In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [2]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1122304/1115394 [==============================] - 3s 3us/step


In [3]:
# 读取并为 py2 compat 解码
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# 文本长度是指文本中的字符个数
print ('Length of text: {} characters'.format(len(text)))

Length of text: 1115394 characters


In [4]:
# 看一看文本中的前 250 个字符
print(text[:250])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [5]:
# 文本中的非重复字符
vocab = sorted(set(text))
print ('{} unique characters'.format(len(vocab)))

65 unique characters


In [6]:
# 创建从非重复字符到索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [7]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  '\n':   0,
  ' ' :   1,
  '!' :   2,
  '$' :   3,
  '&' :   4,
  "'" :   5,
  ',' :   6,
  '-' :   7,
  '.' :   8,
  '3' :   9,
  ':' :  10,
  ';' :  11,
  '?' :  12,
  'A' :  13,
  'B' :  14,
  'C' :  15,
  'D' :  16,
  'E' :  17,
  'F' :  18,
  'G' :  19,
  ...
}


In [8]:
# 显示文本首 13 个字符的整数映射
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'First Citizen' ---- characters mapped to int ---- > [18 47 56 57 58  1 15 47 58 47 64 43 52]


In [9]:
# 设定每个输入句子长度的最大值
seq_length = 100
examples_per_epoch = len(text)//seq_length

# 创建训练样本 / 目标
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

F
i
r
s
t


In [10]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [11]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [12]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 18 ('F')
  expected output: 47 ('i')
Step    1
  input: 47 ('i')
  expected output: 56 ('r')
Step    2
  input: 56 ('r')
  expected output: 57 ('s')
Step    3
  input: 57 ('s')
  expected output: 58 ('t')
Step    4
  input: 58 ('t')
  expected output: 1 (' ')


In [14]:
# 批大小
BATCH_SIZE = 64

# 设定缓冲区大小，以重新排列数据集
# （TF 数据被设计为可以处理可能是无限的序列，
# 所以它不会试图在内存中重新排列整个序列。相反，
# 它维持一个缓冲区，在缓冲区重新排列元素。） 
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
# 词集的长度
vocab_size = len(vocab)

# 嵌入的维度
embedding_dim = 256

# RNN 的单元数量
rnn_units = 1024

In [16]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                batch_input_shape=[batch_size, None]),
      tf.keras.layers.GRU(rnn_units,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [17]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [18]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 65) # (batch_size, sequence_length, vocab_size)


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [20]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [21]:
sampled_indices

array([53, 48,  6, 31,  2,  1, 47, 34, 62, 57, 44, 43, 49, 63, 12,  1, 20,
       45, 27,  9,  5,  8, 15, 60, 18, 44, 18, 32, 17, 22, 38, 16, 59, 60,
       41, 53,  1, 43,  2, 52, 62, 16, 44,  1, 23, 25, 61,  9, 61, 61, 35,
       61, 15, 43, 48, 52, 49, 16, 63, 37, 56,  4, 34, 27,  4, 10, 58, 17,
       29, 10, 27, 61, 51, 43, 61, 61, 41, 25, 45, 19, 13, 45, 30, 36, 29,
       46, 48, 48, 59, 39, 35, 34, 41, 35, 56, 43,  9, 37, 22, 47],
      dtype=int64)

In [22]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 "tion, so he might\nBe call'd your vanquisher.\n\nCORIOLANUS:\nAt Antium lives he?\n\nLARTIUS:\nAt Antium.\n\n"

Next Char Predictions: 
 "oj,S! iVxsfeky? HgO3'.CvFfFTEJZDuvco e!nxDf KMw3wwWwCejnkDyYr&VO&:tEQ:OwmewwcMgGAgRXQhjjuaWVcWre3YJi"


In [23]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 65)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.173446


In [24]:
model.compile(optimizer='adam', loss=loss)

In [25]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [26]:
EPOCHS=10

In [27]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
172/172 [==============================] - 17s 96ms/step - loss: 2.6847
Epoch 2/10
172/172 [==============================] - 15s 85ms/step - loss: 1.9771
Epoch 3/10
172/172 [==============================] - 15s 85ms/step - loss: 1.7096
Epoch 4/10
172/172 [==============================] - 15s 86ms/step - loss: 1.5572
Epoch 5/10
172/172 [==============================] - 15s 86ms/step - loss: 1.4659
Epoch 6/10
172/172 [==============================] - 15s 86ms/step - loss: 1.4045
Epoch 7/10
172/172 [==============================] - 15s 86ms/step - loss: 1.3581
Epoch 8/10
172/172 [==============================] - 15s 86ms/step - loss: 1.3189
Epoch 9/10
172/172 [==============================] - 15s 86ms/step - loss: 1.2850
Epoch 10/10
172/172 [==============================] - 15s 87ms/step - loss: 1.2526


In [28]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_10'

In [29]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [30]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            16640     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 65)             66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________


In [31]:
def generate_text(model, start_string):
    # 评估步骤（用学习过的模型生成文本）
  
    # 要生成的字符个数
    num_generate = 1000
  
    # 将起始字符串转换为数字（向量化）
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)
  
    # 空字符串用于存储结果
    text_generated = []
  
    # 低温度会生成更可预测的文本
    # 较高温度会生成更令人惊讶的文本
    # 可以通过试验以找到最好的设定
    temperature = 1.0
  
    # 这里批大小为 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # 删除批次的维度
        predictions = tf.squeeze(predictions, 0)
  
        # 用分类分布预测模型返回的字符
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
  
        # 把预测字符和前面的隐藏状态一起传递给模型作为下一个输入
        input_eval = tf.expand_dims([predicted_id], 0)
  
        text_generated.append(idx2char[predicted_id])
  
    return (start_string + ''.join(text_generated))

In [32]:
print(generate_text(model, start_string=u"ROMEO: "))

ROMEO: All to accounters and your rast.

DAKE OF AUMERLEY:
Romeo! Here is the waning of you
For tronix meet me; and you so shall
he knew with the mode! hear me here!

Second Citizen:
Dear lord?

KATHARINA:
Nay, it is the manlars and Crowbrarethen:
My sick, and what to bring him than thee in.

First Soldier:
So did you shall go with thee.

LARTIUS:
For Lord it,
Looker over TRE VINCENTIO:
The more may stor the devil here, his groving Ipart,
Her last friend nor than I king, s said, to-do
Be cursed. Let thee gone, I?

Shepherd:
By thieve ut.

DUKE VINCENTIO:
And wounds upon me, why come at the peels; which we should joyful sty
a strate is for me fear, tuth sweep; wh
Of thou remain'd as I, my lord, it sleep'd our enwith o'erdarmy through
At loapiness.

First Ourderer:
How shall you now, I would llave men deay.

GEORGELES:
I would not smile.

Shepherd:
All comfort not the greed,
Unformark not, sweet, and let your sorrows is,
Let will rest, past two curst than for it;
And that their kepsom me

In [36]:
model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [37]:
optimizer = tf.keras.optimizers.Adam()

In [38]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
  
    return loss

In [39]:
# 检查点保存至的目录
checkpoint_dir = './training_checkpoints2'

# 检查点的文件名
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [40]:
# 训练步骤
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()
  
    # 在每个训练周期开始时，初始化隐藏状态
    # 隐藏状态最初为 None
    hidden = model.reset_states()
  
    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)
    
        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch+1, batch_n, loss))
  
    # 每 5 个训练周期，保存（检查点）1 次模型
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))
  
    print ('Epoch {} Loss {:.4f}'.format(epoch+1, loss))
    print ('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.174983978271484
Epoch 1 Batch 100 Loss 2.3342409133911133
Epoch 1 Loss 2.1216
Time taken for 1 epoch 14.603587865829468 sec

Epoch 2 Batch 0 Loss 2.1499178409576416
Epoch 2 Batch 100 Loss 1.9137214422225952
Epoch 2 Loss 1.7525
Time taken for 1 epoch 13.236802101135254 sec

Epoch 3 Batch 0 Loss 1.7468795776367188
Epoch 3 Batch 100 Loss 1.609250783920288
Epoch 3 Loss 1.5398
Time taken for 1 epoch 13.287299394607544 sec

Epoch 4 Batch 0 Loss 1.59104585647583
Epoch 4 Batch 100 Loss 1.535347580909729
Epoch 4 Loss 1.5448
Time taken for 1 epoch 13.386630058288574 sec

Epoch 5 Batch 0 Loss 1.4175759553909302
Epoch 5 Batch 100 Loss 1.4179078340530396
Epoch 5 Loss 1.4306
Time taken for 1 epoch 13.910924196243286 sec

Epoch 6 Batch 0 Loss 1.3718539476394653
Epoch 6 Batch 100 Loss 1.343814730644226
Epoch 6 Loss 1.4095
Time taken for 1 epoch 13.394503116607666 sec

Epoch 7 Batch 0 Loss 1.2922817468643188
Epoch 7 Batch 100 Loss 1.3333489894866943
Epoch 7 Loss 1.3170
Time taken